In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import enum
import pickle

## Get Reviews Using ID - api

In [4]:
import requests

url = "https://maroof.sa/42856/10"

payload = {}
headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'Accept': '*/*',
    'DNT': '1',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    #   'Referer': 'https://maroof.sa/42856',
    'Accept-Language': 'en-US,en;q=0.9,fr-FR;q=0.8,fr;q=0.7,ar;q=0.6',
    'Cookie': 'RefreshFilter=; Maroof_CK=u0021Umc5CNHmH380hyqDX9TJzwlgKEJoHSY8wt2zkIzcBDzUQtGtY57NDsPJ2V6TBTbm13sBLCq51XNboQ==; _ga=GA1.2.7146577.1627495036; _gid=GA1.2.671811124.1627495036; _jsuid=2299650498; ASP.NET_SessionId=1hb011r0hkgvngns2htezsab; __RequestVerificationToken=PWwc_E6ZhdKm_LTCfPkTMgkTByTjrsygfl5DwIjUwtYVPKyxbM2MdQKahfrL70TU-mfZ2vg7a0RUE3MGvLsLdyR8a0yj9sL58lag486g6A01; no_tracky_100935705=1; TS01281e28=01ae665f6ea39b0d4b6cb83cad8b5f9bbf44d7e89d5e088de71d3b120ae9ef649c2f1dca8cd296274f1ff81ffba4c9c3249a17c8ea5546c2018cb76ad7f8b2819ceb39b843ed5ae6c23d6bbc86a79c38967333c454f68575018597f24d3ec64d0179a5fb75ca873a221f620150cb341b42c3f1fa86; TSafcb07d9027=085e8b4c78ab200045b6773a912ee3ab5efaf64a34ade18f954adebadb766ed0588ee87ea3805458086f33f21a113000ca0bab95a479e9cde23d49058f3d63f653a7ebbbc37a72a4f93e7dee04f52fb97c506249c4e828aa9d58db10906dcae4; Maroof_CK=!n3pRivcim5lPvIeDX9TJzwlgKEJoHSHbtSNrc0epQrla5CZs1+Z9RcrftrlPW8aLJ5UJKGm3HyT3rw==; RefreshFilter=; TS01281e28=01ae665f6e94529fd00417e27af7aea223f2b1a57b5e088de71d3b120ae9ef649c2f1dca8cd296274f1ff81ffba4c9c3249a17c8ea5546c2018cb76ad7f8b2819ceb39b8432fd5e1c6ac2d2fa8bc40a164bc7e6cb8e8382475737dcf2abd10b6ab5e1cbdc2eb406d5de66dc495dc522b05b6ba42ee; TSafcb07d9027=085e8b4c78ab20002e9f04f6702c0ddc61afba4d988c1d8ed6cc7304779359d980f0ee261659006208c0f3d698113000b62696467b441ed7e93627db4982b24d93efc3a2afb76225d48a2cc8c0a8b02d0e3ea0864990c4bcc4b1b73b3a1e533d'
}

response = requests.request("GET", url, headers=headers, data=payload)

## Get Categories - scraping

In [12]:
categories = []

req = requests.get("https://maroof.sa/")
s = BeautifulSoup(req.content, 'html.parser')
tags = s.find('ul', id='myTab').findAll('a')

for tag in tags:
    category_id = tag["data-businesstype"]
    category_name = tag.find('img')["alt"]
    categories.append(
        {
            "id": category_id,
            "name": category_name
        }
    )
categories

[{'id': '14', 'name': 'تسويق إلكتروني'},
 {'id': '16', 'name': 'تصوير'},
 {'id': '23', 'name': 'مطبخ و مخبوزات'},
 {'id': '24', 'name': 'حلول إلكترونية'},
 {'id': '25', 'name': 'خدمات أكاديمية'},
 {'id': '26', 'name': 'مستلزمات المرأة'},
 {'id': '34', 'name': 'تصميم وطباعة'},
 {'id': '35', 'name': 'تخطيط مناسبات وحفلات'},
 {'id': '39', 'name': 'كوافيرة و تجميل'},
 {'id': '41', 'name': 'إلكترونيات واكسسورات'},
 {'id': '45', 'name': 'أخرى'},
 {'id': '47', 'name': 'السيارات'},
 {'id': '48', 'name': 'العقارات'},
 {'id': '49', 'name': 'أثاث وديكور'},
 {'id': '51', 'name': 'الحرف والصناعات اليدوية'}]

## Get Links from Category - scraping

In [93]:
list_of_businesses = []
list_of_categories = [14, 26, 45] # 41, 51

for category in list_of_categories:
    r = requests.get(
        f'https://maroof.sa/BusinessType/MoreBusinessList?businessTypeId={category}&pageNumber=0')
    data = json.loads(r.content)
    count = data['Count']
    category_list = []

    if count % 10 == 0:
        num_pages = count/10
    else:
        num_pages = int(count/10) + 1

    for i in range(0, 1):
        r = requests.get(
            f'https://maroof.sa/BusinessType/MoreBusinessList?businessTypeId={category}&pageNumber={i}')
        try:
            data = json.loads(r.content)
        except:
            continue
        count = data['Count']
        for b in data['Businesses']:
            id_ = b["Id"]
            category_list.append(id_)

    list_of_businesses.append(
        {
            "category": category,
            "businesses": category_list
        }
    )

list_of_businesses

[{'category': 14,
  'businesses': [205600,
   205599,
   205595,
   205593,
   205585,
   205584,
   205582,
   205580,
   205575,
   205573]},
 {'category': 26,
  'businesses': [205579,
   205570,
   205565,
   205543,
   205540,
   205534,
   205533,
   205524,
   205516,
   205514]},
 {'category': 45,
  'businesses': [205604,
   205598,
   205597,
   205594,
   205592,
   205590,
   205587,
   205586,
   205578,
   205576]}]

## Get Business Details - scraping

In [134]:
all_details = []
for cat in list_of_businesses:
    cat_details = []
    print("Cetgory --->  " , cat)
    for id_ in cat["businesses"]:
        req = requests.get(f"https://maroof.sa/{id_}")
        soup = BeautifulSoup(req.content, "html.parser")
        name = soup.select('div.media.mr-b-10')[0].find('h3').find('span').text
        rating = soup.find('span', class_="rating-num").find_next_sibling().text
        about = soup.select('div.panel-body.height-421')[0].find('div', class_="col-xs-12").text
        socials = soup.select("div.panel-body.height-421")[1].find('div', class_='wrraper')
        socials = socials.findAll('div', class_='col')

        list_of_socials = []

        for social in socials:
            try:
                key_ = ""
                if "instagram" in social.findAll('p')[1].text:
                    key_ = "instagram"
                elif "twitter" in social.findAll('p')[1].text:
                    key_ = "twitter"
                elif "facebook" in social.findAll('p')[1].text:
                    key_ = "facebook"
                elif "snapchat" in social.findAll('p')[1].text:
                    key_ = "snapchat"
                elif "موقع" in social.findAll('p')[0].text or "salla" in social.findAll('p')[1].text:
                    key_ = "website"
                
                print(key_)
                
                list_of_socials.append(
                {
                    key_ : social.findAll('p')[1].text
                }
            )
            except:
                pass

        contacts = soup.select("div.panel-body.height-455")[0]
        contacts = contacts.findAll('div', class_='social-row')

        for contact in contacts:
            try:
                key_ = ""
                if "رقم" in contact.findAll('p')[0].text:
                    key_ = "phone"
                elif "@" in contact.findAll('p')[1].text:
                    key_ = "email"
                elif "واتس" in contact.findAll('p')[0].text:
                    key_ = "whatsapp"
                elif "عنوان" in contact.findAll('p')[0].text:
                    key_ = "address"
                elif "t.me" in contact.findAll('p')[1].text:
                    key_ = "telegram"
                
                print(key_)
                    
                list_of_socials.append(
                {
                    key_ : contact.findAll('p')[1].text
                }
            )
            except:
                print("probleeeem")
                pass

        item = {
            "category" : cat['category'],
            "name" : name,
            "rating" : rating,
            "about" : about,
        }

        for social in list_of_socials:
            k, v = list(social.items())[0]
            item[k] = v
        
        cat_details.append(item)
        print(name , " Scraped !!")
        
    all_details.append(cat_details)

Cetgory --->   {'category': 14, 'businesses': [205600, 205599, 205595, 205593, 205585, 205584, 205582, 205580, 205575, 205573]}
website
phone
email
whatsapp
إيفانوشوب  Scraped !!
website
phone
email
whatsapp
افضل اختيار  Scraped !!
instagram
phone
email
whatsapp
فصول للعطور والبخور  Scraped !!
twitter
phone
email
whatsapp
telegram
فـــايست ببجي  Scraped !!
instagram
phone
email
سعودي ماركيتنق  Scraped !!
website
instagram
twitter
phone
email
whatsapp
telegram
address
همسات رنيم  Scraped !!
website
phone
email
whatsapp
آي فور يو  Scraped !!
website

phone
email
whatsapp
متجر سيلين لمستلزمات المنزل والاسرة  Scraped !!

phone
email
whatsapp
telegram
متجر ايتونز للبطايق الرقميه  Scraped !!
website

phone
phone
email
ترافوتلز  Scraped !!
Cetgory --->   {'category': 26, 'businesses': [205579, 205570, 205565, 205543, 205540, 205534, 205533, 205524, 205516, 205514]}
website
phone
email
whatsapp
متجر جولز   Scraped !!
instagram
instagram
phone
email
متجر ضمة  Scraped !!
instagram
phone
email
سل

In [135]:
all_details

[[{'category': 14,
   'name': 'إيفانوشوب',
   'rating': '0 ',
   'about': '\r\n                بيع مستلزمات ألاسرة والمنزل والجمال\r\n            ',
   'website': 'https://ivanoshop.com',
   'phone': '0551499886',
   'email': ' ivanoshopsa@gmail.com ',
   'whatsapp': '966551499886'},
  {'category': 14,
   'name': 'افضل اختيار',
   'rating': '0 ',
   'about': '\r\n                بيع وتسويق\r\n            ',
   'website': 'https://salla.sa/salla.saboxto ...',
   'phone': '0563297777',
   'email': ' abdullatiff9999@gmail.com ',
   'whatsapp': '966563297777'},
  {'category': 14,
   'name': 'فصول للعطور والبخور',
   'rating': '0 ',
   'about': '\r\n                بيع العطور والبخور\r\n            ',
   'instagram': 'https://www.instagram.com/seas ...',
   'phone': '0599980987',
   'email': ' 1989Mujtaba@gmail.com ',
   'whatsapp': '966599980987'},
  {'category': 14,
   'name': 'فـــايست ببجي',
   'rating': '10 ',
   'about': '\r\n                بيع حسابات ببجي ومنصة وساطة\r\n            

In [136]:
flat_list = [item for sublist in all_details for item in sublist]
dataframe = pd.DataFrame(flat_list)
dataframe

,category,name,rating,about,website,phone,email,whatsapp,instagram,twitter,telegram,address,,facebook
0,14,إيفانوشوب,0,\r\n بيع مستلزمات ألاسرة والمنز...,https://ivanoshop.com,0551499886,ivanoshopsa@gmail.com,966551499886,NaN,NaN,NaN,NaN,NaN,NaN
1,14,افضل اختيار,0,\r\n بيع وتسويق\r\n,https://salla.sa/salla.saboxto ...,0563297777,abdullatiff9999@gmail.com,966563297777,NaN,NaN,NaN,NaN,NaN,NaN
2,14,فصول للعطور والبخور,0,\r\n بيع العطور والبخور\r\n ...,NaN,0599980987,1989Mujtaba@gmail.com,966599980987,https://www.instagram.com/seas ...,NaN,NaN,NaN,NaN,NaN
3,14,فـــايست ببجي,10,\r\n بيع حسابات ببجي ومنصة وساط...,NaN,0561870214,Saleh.i96@hotmail.com,966561870214,NaN,https://twitter.com/FTI010,https://t.me/FTI01,NaN,NaN,NaN
4,14,سعودي ماركيتنق,0,\r\n عرض قطع الماركات الأصلية ف...,NaN,0549260602,saudimarkiting@gmail.com,NaN,https://www.instagram.com/saud ...,NaN,NaN,NaN,NaN,NaN
5,14,همسات رنيم,0,\r\n بيع منجات التجميل والاكسسو...,https://salla.sa/hamasatraneem ...,0503003540,hamasat.raneem@hotmail.com,966503003540,https://instagram.com/hamasat. ...,https://twitter.com/Hamasatran ...,https://t.me/hamasatraneem,\r\n 7505 ش...,NaN,NaN
6,14,آي فور يو,0,\r\n متجر الكتروني\r\n,https://salla.sa/w4u/,0530048300,xshybh@gmail.com,966530048300,NaN,NaN,NaN,NaN,NaN,NaN
7,14,متجر سيلين لمستلزمات المنزل والاسرة,0,\r\n متجر اللكتروني كل مايخص مس...,https://celeini.com/,0563396069,moayad7277@gmail.com,9665633096069,NaN,NaN,NaN,NaN,https://celeini.com/,NaN
8,14,متجر ايتونز للبطايق الرقميه,0,\r\n بيع بطاقات ايتونز وجوجل بل...,NaN,0568978696,naef.store@gmail.com,966568978696,NaN,NaN,https://t.me/itunaef,NaN,https://t.me/fcuuuv,NaN
9,14,ترافوتلز,0,\r\n نقدم جميع خدمات السفر والس...,https://www.travotels.com,920004262,w.nassar@gama-ksa.com,NaN,NaN,NaN,NaN,NaN,https://www.travotels.com,NaN
